In [1]:
import tensorflow as tf #importing libraries 
import pandas as pd #data frame
import numpy as np #array
import matplotlib.pyplot as plt #visualization
import nltk #nlp lib
import re #lower case
from nltk.corpus import stopwords #unwanted words lib(stopwords)
from nltk.stem.porter import PorterStemmer #stemming
from sklearn.model_selection import train_test_split #dividing data sets train test 
from tensorflow.keras.layers import Dropout #droup out layer
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
from tensorflow.keras.layers import Embedding #embidding layer lib
from tensorflow.keras.preprocessing.sequence import pad_sequences #padding post 0-0-0
from tensorflow.keras.models import Sequential #sequential model lib
from tensorflow.keras.preprocessing.text import one_hot #one hot represnet 
from tensorflow.keras.layers import LSTM #lstm lib
from tensorflow.keras.layers import Dense 
from tensorflow.keras.layers import Bidirectional

In [2]:
datatrain=pd.read_csv("/content/drive/My Drive/Colab Notebooks/fakenewslstm/train.csv") #loading datasets  

In [ ]:
datatrain.head(3)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1


In [ ]:
datatrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [3]:
datatrain.dropna(inplace=True) #droping nan values
datatrain.reset_index(inplace=True) #reseting index after droping
x=datatrain["title"] #dependent feature storing in X
print(x.shape)
x.head(3)
y=datatrain["label"] #independent feature
print(y.shape)
y.head(3)

(18285,)
(18285,)


0    1
1    0
2    1
Name: label, dtype: int64

In [4]:
nltk.download("stopwords") #loading stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
ps = PorterStemmer() #steming processes
corpus = []
for i in range(0, len(x)): #till the length of X (means text)
    review = re.sub('[^a-zA-Z]', ' ', x[i]) #special word apart from A-Z||a-z will be replaced by " " in text field
    review = review.lower() # lowering  letter
    review = review.split() #spliting each word
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')] #removing stop waords
    review = ' '.join(review)
    corpus.append(review)

In [6]:
voc_size=5000 # intializing dict size
onehot_repr=[one_hot(words,voc_size)for words in corpus] #performing one hot encoder after tokenizibg the text field to sentence->stemming ->stop word
sent_length=20 ## creating max length of each sentence
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length) #padding zero if length is less than 20

In [8]:
"""embedding_vector_features=40  # dimension of feature
model=Sequential() #model creation
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) #adding embedding layer
model.add(LSTM(100)) #adding lstm layer
model.add(Dense(1,activation='sigmoid')) #output layer
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())"""

"embedding_vector_features=40  # dimension of feature\nmodel=Sequential() #model creation\nmodel.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) #adding embedding layer\nmodel.add(LSTM(100)) #adding lstm layer\nmodel.add(Dense(1,activation='sigmoid')) #output layer\nmodel.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])\nprint(model.summary())"

In [11]:
embedding_vector_features=40 # dimension of feature
model1=Sequential() #model creation
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) #adding embedding layer
model1.add(Bidirectional(LSTM(100))) #bidirection layer
model1.add(Dropout(0.3)) #drop out layer
model1.add(Dense(1,activation='sigmoid')) #output layer
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
len(embedded_docs),y.shape

(18285, (18285,))

In [14]:
xfinal=np.array(embedded_docs)
yfinal=np.array(y)

In [15]:
print(xfinal.shape,yfinal.shape)

(18285, 20) (18285,)


In [16]:
xtrain, xtest, ytrain, ytest = train_test_split(xfinal, yfinal, test_size=0.33, random_state=42)

In [17]:
model1.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=1,batch_size=64)

192/192 [==============================] - 2s 13ms/step - loss: 0.3053 - accuracy: 0.8522 - val_loss: 0.2025 - val_accuracy: 0.9150


In [18]:
ypred=model1.predict_classes(xtest)
ypred

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]], dtype=int32)

In [19]:
confusion_matrix(ytest,ypred)

array([[2963,  456],
       [  57, 2559]])

In [21]:
accuracy_score(ytest,ypred)

0.9149958574979288